In [2]:
# Define the important variables here
import json
import os
import pandas as pd

# Import the main functions 
from ipynb.fs.full.data_fill_up import *

target = {
    "fips": 6093,
    "target_county_name": "Siskiyou",
    "date_column_name": "DATE",
    "datetime_format": "%Y%m%d",
    "file_name": "",
    "target_data_field_name": "prcp"
}
target['file_name'] = f"Datasets/merged_tp_precip_wind_fmc_{target['target_county_name']}.csv"

# Use the adjacent county of the target county to find referral data like aws or fmc
# Use the following link to find the adjacent county
# https://gis.data.ca.gov/datasets/8713ced9b78a4abb97dc130a691a8695/explore?location=39.765076%2C-121.456785%2C8.00

# prcp
prcp_referral = {
    "fips_code": 6093,
    "referral_county_name": "Siskiyou",
    "fips_column_name": "county_fips",
    "date_column_name": "date",
    "datetime_format": "%Y%m%d",
    "file_name": "Datasets/tp_zipcode_county.csv",
    "data_fields_to_be_copied": ["prcp", "tmax", "tmin", "tavg"]
}

# Wind aws
wind_aws_referral = {
    "fips_code": 6093,
    "referral_county_name": "Siskiyou",
    "fips_column_name": "fips",
    "date_column_name": "date",
    "datetime_format": "%Y-%m-%d",
    "file_name": "Datasets/wind_with_fips.csv",
    "data_fields_to_be_copied": ["aws"]
}

# fuel moisture content 
fmc_referral = {
    "fips_code": 6093,
    "referral_county_name": "Siskiyou",
    "fips_column_name": "fips",
    "date_column_name": "date",
    "datetime_format": "%Y-%m-%d",
    "file_name": "Datasets/fuel_with_fips.csv",
    "data_fields_to_be_copied": ["percent"]
}

# Put the county target here
#my_fips = 6093
#my_county_name = "Siskiyou"
#my_fips = 6105
#my_county_name = "Trinity"
#my_fips = 4005
#my_county_name = "Coconino"
#my_fips = 16035
#my_county_name = "Clearwater"
#my_fips = 16085
#my_county_name = "Valley"
#my_fips = 4015
#my_county_name = "Mohave"
#my_fips = 2290
#my_county_name = "Yukon-Koyukuk"
my_fips = 16049
my_county_name = "Idaho"

# Config

def setup_county_config(my_fips, my_county_name):
    target["fips"] = my_fips
    target["target_county_name"] = my_county_name
    target['file_name'] = f"Datasets/merged_tp_precip_wind_fmc_{my_fips}_{target['target_county_name']}.csv"

    prcp_referral["fips_code"] = my_fips
    prcp_referral["referral_county_name"] = my_county_name
    wind_aws_referral["fips_code"] = my_fips
    wind_aws_referral["referral_county_name"] = my_county_name
    fmc_referral["fips_code"] = my_fips
    fmc_referral["referral_county_name"] = my_county_name

def generate_one():
    generate_csv_for_fips(target)
    if ( merge_data_referral_to_target(target, prcp_referral) > 0) :
        if ( merge_data_referral_to_target(target, wind_aws_referral) > 0):
            merge_data_referral_to_target(target, fmc_referral)
    drop_ana(target)

def get_row_count(csv_file_name):
        df = pd.read_csv(csv_file_name)
        return df.shape[0]

def generate_all(tracing_file):
    status_tracing_df = pd.read_csv(tracing_file)
    # Loop through the dataframe and change the column value on each row
    for i in range(len(status_tracing_df)):
        # Check if it is 1, means to generate; otherwise skip this row
        if ( status_tracing_df.loc[i, 'to_generate'] != 1):
            continue
        print(f"### Generating data for {status_tracing_df.loc[i, 'fips']}, {status_tracing_df.loc[i, 'county_name']}")
        # Setup a config for the county
        setup_county_config( status_tracing_df.loc[i, 'fips'], status_tracing_df.loc[i, 'county_name'])
        # Generate the data for the county
        generate_one()
        # Save the status 
        status_tracing_df.loc[i, 'status'] = 1
        status_tracing_df.loc[i, 'to_generate'] = 0
        row_count = get_row_count(target['file_name'])
        status_tracing_df.loc[i, 'generated_row_count'] = row_count
        if (row_count <= 0):
            # Delete the file if no rows
            os.remove(target['file_name'])
            status_tracing_df.loc[i, 'file_name'] = ''
        else:
            status_tracing_df.loc[i, 'file_name'] = target['file_name']
        save_df = status_tracing_df.copy()
        save_df.drop(save_df.filter(regex="Unname"),axis=1, inplace=True)
        save_df.to_csv( tracing_file)
#
# Main program
#
#  
tracing_file = "Datasets/data_generation_status.csv"
generate_all(tracing_file)

### Generating data for 37141, Pender County


/Users/billibit/Projects/wildfire.paper/data_fill_up.ipynb:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    print(f\"The file {target['file_name']} contains data {target_dataset.shape}\")\n",


Saved to Datasets/merged_tp_precip_wind_fmc_37141_Pender County.csv
The file Datasets/merged_tp_precip_wind_fmc_37141_Pender County.csv contains data (0, 15)


/Users/billibit/Projects/wildfire.paper/data_fill_up.ipynb:27: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  "    filtered_rows = df[df[column_name] == value_to_find]\n",


The file Datasets/tp_zipcode_county.csv contains data (8438850, 16)
The county 37141 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37019 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37047 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37061 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37129 has 2678 rows of data in the Datasets/tp_zipcode_county.csv
The county 37133 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37141 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37163 has 0 rows of data in the Datasets/tp_zipcode_county.csv
*** Using the adjacent county New Hanover County, 37129 of Pender County. It has 2678 rows of data in the Datasets/tp_zipcode_county.csv
referral_dataset with fips 37129 = (2678, 16)
Saved to Datasets/merged_tp_precip_wind_fmc_37141_Pender County.csv
The file Datasets/merged_tp_precip_wind_fmc_37141_Pender County.csv contains data (0, 

...

The file Datasets/tp_zipcode_county.csv contains data (8438850, 16)
The county 37149 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37149 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37161 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 45045 has 2276 rows of data in the Datasets/tp_zipcode_county.csv
The county 45083 has 3681 rows of data in the Datasets/tp_zipcode_county.csv
*** Using the adjacent county Spartanburg County, 45083 of Polk County. It has 3681 rows of data in the Datasets/tp_zipcode_county.csv
referral_dataset with fips 45083 = (3681, 16)
Saved to Datasets/merged_tp_precip_wind_fmc_37149_Polk County.csv
The file Datasets/merged_tp_precip_wind_fmc_37149_Polk County.csv contains data (0, 15)
Saved to Datasets/merged_tp_precip_wind_fmc_37149_Polk County.csv
### Generating data for 37151, Randolph County


/Users/billibit/Projects/wildfire.paper/data_fill_up.ipynb:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "\n",
/Users/billibit/Projects/wildfire.paper/data_fill_up.ipynb:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    fire_data = pd.read_csv(\"Datasets/aggregated_wildfire.csv\")\n",
/Users/billibit/Projects/wildfire.paper/data_fill_up.ipynb:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

Saved to Datasets/merged_tp_precip_wind_fmc_37151_Randolph County.csv
The file Datasets/merged_tp_precip_wind_fmc_37151_Randolph County.csv contains data (14, 15)


/Users/billibit/Projects/wildfire.paper/data_fill_up.ipynb:27: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  "    filtered_rows = df[df[column_name] == value_to_find]\n",


The file Datasets/tp_zipcode_county.csv contains data (8438850, 16)
The county 37151 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37037 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37057 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37081 has 3681 rows of data in the Datasets/tp_zipcode_county.csv
The county 37123 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37125 has 0 rows of data in the Datasets/tp_zipcode_county.csv
The county 37151 has 0 rows of data in the Datasets/tp_zipcode_county.csv
*** Using the adjacent county Guilford County, 37081 of Randolph County. It has 3681 rows of data in the Datasets/tp_zipcode_county.csv
referral_dataset with fips 37081 = (3681, 16)
Add data to the original 2015-02-14 00:00:00 <- 20150214  delta days 0: 
prcp = 0.0
tmax = 111.0
tmin = -71.0
tavg = -1.0
Add data to the original 2015-03-31 00:00:00 <- 20150331  delta days 0: 
prcp = 0.0
tmax = 233.0
tmin = 33.